# Qubo

## Chemistry Example

In [0]:
# import stuff
from collections import OrderedDict
import itertools
import numpy as np
from pprint import pprint
from dadk.FujitsuEmulatedDASolver import *
emda_solver = FujitsuEmulatedDASolver()

number of GPU available: 0


In [0]:
# set up nested (ordered) Dict

# custom class
class IndexingArray(OrderedDict):
    def __missing__(self, key):
        val = self[key] = IndexingArray()
        return val
    
groups = {'H':70, 'CH3':15, 'CF3':69, 'OH':17, 'NH2':16, 'CH2CH3':71}
# groups = {'H':1, 'CH3':1, 'CF3':2, 'OH':1, 'NH2':1, 'CH2CH3':2}
R0 = np.array(['H', 'CH3', 'CF3', 'OH'])
R2 = np.array(['H', 'CH3', 'CF3', 'OH'])
R1 = np.array(['H', 'CH3', 'NH2', 'CH2CH3'])
R3 = np.array(['H', 'CH3', 'NH2', 'CH2CH3'])
items = [R0, R1, R2, R3]
    
# populate dict
d = IndexingArray()
k=0
for i, r in enumerate([R0, R1, R2, R3]):
    for j, g in enumerate(r):
        d['R'+str(i)][g] = k
        k += 1
        
pprint(d)
pprint(d['R0']['CF3'])

IndexingArray([('R0',
                IndexingArray([('H', 0), ('CH3', 1), ('CF3', 2), ('OH', 3)])),
               ('R1',
                IndexingArray([('H', 4),
                               ('CH3', 5),
                               ('NH2', 6),
                               ('CH2CH3', 7)])),
               ('R2',
                IndexingArray([('H', 8), ('CH3', 9), ('CF3', 10), ('OH', 11)])),
               ('R3',
                IndexingArray([('H', 12),
                               ('CH3', 13),
                               ('NH2', 14),
                               ('CH2CH3', 15)]))])
2


In [0]:
# Objective function: max x1 + x2 + x3 + x4
def H_obj(d):
    H = BinPol()
    for k, v in d.items():
        for group, idx in v.items():
            H.set_term(groups[group], (idx,))
    return H

# print(H_obj(d))

In [0]:
# Constraint A: select exactly one item -> P(1-x-y+2xy)
def H_c1(d, P, r):
    p = BinPol().set_term(1,())
    list = [(group, idx) for group, idx in d[r].items()]
    for pair in itertools.combinations(list, r=2):
        p.set_term(-1,(pair[0][1],))
        p.set_term(-1,(pair[1][1],))
    p.multiply(p)
    p.multiply_scalar(P) 
    return p

# print(H_c1(d, r='R0', P=1))

In [0]:
# Constraint B: Ri and Rj cannot have the same group -> P(xy)
def H_c2(d, P, ri, rj):
    p = BinPol()
    list1 = [(group, idx) for group, idx in d[ri].items()]
    list2 = [(group, idx) for group, idx in d[rj].items()]
    for pair1 in list1:
        for pair2 in list2:
            if pair1[0] == pair2[0]:
                #print(pair1, pair2)
                p.set_term(1, (pair1[1], pair2[1]))
    p.multiply_scalar(P) 
    return p
                           
# print(H_c2(d, ri='R1', rj='R2', P=1))

In [0]:
# combine
def H_all(d, P):
    H = BinPol()
    H.add(H_obj(d))
    H.add(H_c1(d, P, r='R0').multiply_scalar(-1)) # multiply by -1 to subtract instead of adding
    H.add(H_c1(d, P, r='R1').multiply_scalar(-1)) # same as above
    H.add(H_c1(d, P, r='R2').multiply_scalar(-1)) # same as above
    H.add(H_c1(d, P, r='R3').multiply_scalar(-1)) # same as above
    H.add(H_c2(d, P, ri='R0', rj='R1').multiply_scalar(-1))
    H.add(H_c2(d, P, ri='R2', rj='R3').multiply_scalar(-1))
    H.add(BinPol().set_term(P, (d['R0']['CF3'], )).set_term(-P, (d['R2']['CF3'], )))
    H.add(BinPol().set_term(P, (d['R1']['CH2CH3'], )).set_term(-P, (d['R3']['CH2CH3'], )))
    return H

# print(H_all(d, P=1))

In [0]:
# take a look at the final polynomial
print('Subtract constraint polynomial from Objective function:\n {}'
      .format(H_all(d, P=1)))

Subtract constraint polynomial from Objective function:
  - 4 + 71 x_0 - 2 x_0 x_1 - 2 x_0 x_2 - 2 x_0 x_3 - 1 x_0 x_4 + 16 x_1 - 2 x_1 x_2 - 2 x_1 x_3 - 1 x_1 x_5 + 71 x_2 - 2 x_2 x_3 + 18 x_3 + 71 x_4 - 2 x_4 x_5 - 2 x_4 x_6 - 2 x_4 x_7 + 16 x_5 - 2 x_5 x_6 - 2 x_5 x_7 + 17 x_6 - 2 x_6 x_7 + 73 x_7 + 71 x_8 - 2 x_8 x_9 - 2 x_8 x_10 - 2 x_8 x_11 - 1 x_8 x_12 + 16 x_9 - 2 x_9 x_10 - 2 x_9 x_11 - 1 x_9 x_13 + 69 x_10 - 2 x_10 x_11 + 18 x_11 + 71 x_12 - 2 x_12 x_13 - 2 x_12 x_14 - 2 x_12 x_15 + 16 x_13 - 2 x_13 x_14 - 2 x_13 x_15 + 17 x_14 - 2 x_14 x_15 + 71 x_15


In [0]:
# Solve It ...

# try different values for P
for P in range(1,100, 10) :
    H = H_all(d, P)
    H.multiply_scalar(-1) # multiply by -1 to maximize instead of minimize

    emda_sol = emda_solver.minimize(H)
    x = emda_sol.get_minimum_energy_solution().configuration
    cnt_selected = np.count_nonzero(x)
    if not cnt_selected == 4:
        print('P={} {} -> invalid solution'.format(P, x))
    else:
        print('P={} -> valid solution: {} value: {}'.format(P, x, H.compute(x)))
        selected = [list(np.concatenate(items))[m] for m in np.nonzero(x)[0]]  
        weights = [groups[g] for g in selected]
        print("Selected molecules and their weights:\n {} {}. Total weight: {}".format(selected, weights, sum(weights)))
        break

P=1 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] -> invalid solution
P=11 [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1] -> invalid solution
P=21 [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1] -> invalid solution
P=31 [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1] -> invalid solution
P=41 [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1] -> invalid solution
P=51 [0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1] -> invalid solution
P=61 [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0] -> invalid solution
P=71 -> valid solution: [0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0] value: -369
Selected molecules and their weights:
 ['CF3', 'CH2CH3', 'OH', 'H'] [69, 71, 17, 70]. Total weight: 227
